
<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color: gold ; color : 'black'; text-align: left; border-radius: 2px 2px;pad:4px;"> Problem Statement </h2>

<strong>Problem Statement:</strong>

We have a lot of customer on our InvestorAI platform. Whenever they login in our app and view anything, we get pings from their mobile phone indicating that they are using the app.
<br>
You have been provided with 3 weeks of training data and 1 week of test data.
<br>
<br>
<strong>About Data and Variables</strong>

Training data contains id, gender, age, number of kids the customer has and all the pings that have been received (during the training data period).
<br><br>
<strong>Objective:</strong>

We want to predict how many hours the customer will be online / using our app on a given day. So the test data contains customer id, and date (during the test data period).
The test data also contains the actual online hours, which is what your model should predict.

<strong>Evaluation Metric:</strong>

We will be looking at Root Mean Squared Error or RMSE for short (lower the better) to see how good your model is.

<strong> Special Note:</strong>

We have a separate held out test dataset against which we will evaluate your model for overfitting, underfitting, etc.

<strong>Initial Insights: </strong>

1. Looks like we are handling a raw data as custumer database and the activity are seperated out in the data itself. So few assumpitions and data cleaning could be needed ?
2. Clearly given in the problem statement that we are going to handle a time related data for each customer.  This could be a multivariante time series problem with multiple timeseries. So modeling stategy and model building need to  be done to capture patterns, ( lets assume)
3. Error metric is root mean squred error so its sensitive to small changes, can I optimize my model for better metric, root mean squared log error which is more reponsive to very fine changes?
4. As there is another secret hold out set for validation for overfitting and underfitting, so can I hold few datapoints to check this prior?

<p style = "font-size:25px"><strong>Lets Dive Into The Data....</strong></p>


<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color: gold ; color : 'black'; text-align: left; border-radius: 2px 2px;pad:4px;"> Data and Initial Stats </h2>

loading various module I am planning to use for this assignment

In [1]:
#importing modules

import warnings 
warnings.filterwarnings('ignore')

import time
t = time.time()

print('Importing startred...')

# base libraries
import os
import numpy as np
import pandas as pd
import re
from scipy import stats
from random import randint
from datetime import datetime


# visualization libraries
import matplotlib.pyplot as plt
import matplotlib 
%matplotlib inline
import seaborn as sns
import missingno as msno


# preprocessing libraries
from sklearn.model_selection import (TimeSeriesSplit,
                                     GridSearchCV,
                                     RandomizedSearchCV)

from sklearn.preprocessing import (LabelEncoder,
                                   StandardScaler, 
                                   MinMaxScaler, 
                                   OrdinalEncoder)

from sklearn.feature_selection import SelectFromModel


# metrics
from sklearn.metrics import (mean_squared_error, 
                             r2_score, 
                             mean_absolute_error)
from sklearn.metrics import make_scorer


# modeling algos
from sklearn.linear_model import (LogisticRegression,
                                  Lasso, 
                                  ridge_regression,
                                  LinearRegression)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (AdaBoostRegressor, 
                              RandomForestRegressor,
                              VotingRegressor, 
                              GradientBoostingRegressor)
from xgboost import XGBRegressor
from lightgbm import (LGBMRegressor,
                      early_stopping)

from sklearn.base import clone ## sklearn base models for stacked ensemble model


#Interpretiability of the model
import shap
import eli5
from eli5.sklearn import PermutationImportance


## misea
from sklearn.pipeline import make_pipeline



print('Done,All the required modules are imported. Time elapsed: {}sec'.format(time.time()-t))

Importing startred...


Using TensorFlow backend.


Done,All the required modules are imported. Time elapsed: 12.516072988510132sec


<p style = "font-size:25px"><strong>Data Loading and Visualization....</strong></p>

In [3]:
# loading data

customer_df = pd.read_csv('./data/customers.csv', delimiter = ',', encoding = 'utf-8')
pings_df = pd.read_csv('./data/pings.csv', delimiter = ',', encoding = 'utf-8')
test_df = pd.read_csv('./data/test.csv', delimiter = ',', encoding = 'utf-8')

<p style = "font-size:20px"><strong>Sneak peak into the data....</strong></p>

In [14]:
print('*'*10 + 'Data Summary and Inital peaks' + '*'*10 )

print('\n'+'*'*15 + 'Shapes of Data' + '*'*15+'\n' )

print('Shape of the Customer database: {}'.format(customer_df.shape))
print('Shape of the Pings dataset: {}'.format(pings_df.shape))
print('Shape of the Test dataset: {}'.format(test_df.shape))

print('\n'+'*'*50 + '\n')

print('Head of Customer database'+ '\n')
print(customer_df.head())

print('\n' +'*'*50+ '\n')

print('Head of Ping dataset'+ '\n')
print(pings_df.head())

print('\n' +'*'*50 + '\n')

print('Head of Test dataset'+ '\n')
print(test_df.head())

print('\n' +'*'*50 + '\n')


**********Data Summary and Inital peaks**********

***************Shapes of Data***************

Shape of the Customer database: (2500, 4)
Shape of the Pings dataset: (50528701, 2)
Shape of the Test dataset: (17500, 3)

**************************************************

Head of Customer database

       id gender  age  number_of_kids
0  979863   MALE   26               2
1  780123   MALE   60               2
2  614848   MALE   45               4
3  775046   MALE   62               3
4  991601   MALE   23               0

**************************************************

Head of Ping dataset

       id   timestamp
0  899313  1496278800
1  373017  1496278800
2  798984  1496278800
3  245966  1496278800
4  689783  1496278800

**************************************************

Head of Test dataset

       id      date  online_hours
0  979863  28/06/17             7
1  979863  27/06/17             9
2  979863  26/06/17             9
3  979863  25/06/17            10
4  979863  24/06/17  

<strong>Initial Insights about Data</strong>
1. As given in problem statement we have three csv files, customer, pings, test datasets. All in common is uniques id, Its customer id.
2. Customer database gives all the unique things about customer. This include children, gender, age, of the customer.
3. Coming to Ping dataset, data of timestamp is little difficult to compehend as its in unix epoch encodoing, as it is number of seconds from 1970s to specific time, need conversion to make something out of it.
4. Test data giving some hints, how I need to convert my training dataset? May be to datatime object.


Now questions are how to convert ping data timestamp form epoch timestamp to pandas timestamp? and how to calculate the online hours like test dataset? and finaly need to merge customer dataset to ping dataset and test dataset..


first thing first... lets convert ping data timestep to datatime timestamp and extract date, and other basic things later. 

<strong>Number of Online Hours Calculation Assumption: </strong>

Here is the thing, I can calculate online hours directly by taking the one step slide diffrence of timestamps for each date for each customer (by grouping them together). But logically this could be a flawed approch. Reason behind this is no idea when session of custemer end as there is no capping limit given in problem statement. 

So, assuming if the slide time difference is less than 2minutes (i.e, 120Sec)
user active online time is same, if its more than that lets cap it to 2mins.


lets implement this idea to the ping dataset... 

In [ ]:
### value sorting with respect to id and timestamp
pings_df = pings_df.sort_values(by = ['id','timestamp']).reset_index(drop=True)

# creating a copy to preserve actual ping data
temp_ping_df = pings_df.copy()

#pre-processing data
temp_ping_df.drop_duplicates(inplace = True)
temp_ping_df['timestamp_decode'] = temp_ping_df['timestamp'].apply(lambda x: datetime.fromtimestamp(x))



## timestamp and datagrouping
temp_ping_df['date'] = temp_ping_df['timestamp_decode'].dt.date
temp_ping_df['online_hours'] = (temp_ping_df.groupby(by=['id','date'])['timestamp'].diff())/(60*60)
temp_ping_df['online_hours']  =  temp_ping_df['online_hours'].apply(lambda x: x if x< (2/60) else (2/60))

temp_ping_df.fillna(0,inplace = True)


#### creating our training data
train_df= (temp_ping_df.groupby(by = ['id','date'])['online_hours'].sum()).reset_index()
train_df['online_hours'] = round(train_df['online_hours'],1)
train_df.head()